### PREGUNTA 4

El archivo Web Server Requests G contiene un conjunto de datos que proporciona un registro histórico (sintético) de la tasa de llegada de solicitudes que un servidor web ha experimentado durante un intervalo de tiempo de 5 minutos. Esta información es esencial para comprender y gestionar la carga del servidor, planificar la infraestructura, y garantizar un rendimiento óptimo
en un entorno de servidor web real.
1. Identificar patrones de tráfico web como horas pico de actividad y momentos de menor demanda.
2. Ajuste una distribución gamma a sus datos de “Arrival Rate”, trace el histograma de sus datos junto con la función de densidad de probabilidad (pdf) de la distribución gamma ajustada y muestre los par´ametros de la distribución gamma ajustada.
3. ¿Cuál es la probabilidad de que el servidor reciba más de 8 solicitudes durante el próximo intervalo?
4. ¿Cuál es la probabilidad de que la tasa de llegada supere un umbral crítico de, por ejemplo, 15 solicitudes por minuto, en el próximo intervalo?
5. Si la capacidad máxima de procesamiento del servidor es de 25 solicitudes por intervalo, ¿Cuál la probabilidad de superar la capacidad máxima de procesamiento del servidor en el siguiente intervalo?

In [3]:
import pandas as pd

df = pd.read_csv('../data/Web_Server_Requests_G.csv')